In [1]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [2]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

D:\pycharm_project\vs2ttsvoice\encoder\audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder\saved_models\pretrained\pretrained.pt


In [29]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed,_ = encoder.embed_utterance(preprocessed_wav)
    return embed

In [25]:
class X2E_NetL1LNO(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(X2E_NetL1LNO, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))  
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [26]:
#模型
model=X2E_NetL1LNO(512,400,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
# criterion = torch.nn.MSELoss(reduction='sum')
criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [27]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\X2E'
#损失值写入文件
se_dir='D:\\pycharm_project\\vs2ttsvoice\\SE_vector'
for i in range(80):
    print(i)
    pathstr ='xvector'+str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir,pathstr)
    sepath='dpdir'+str(i+1)+'.txt' 
    sepath=os.path.join(se_dir,sepath)
    se_vectors=open(sepath).readlines()
    file=open( pathstr)
    j=0
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        se=se_vectors[j].replace('[','').replace(']','').split()
        se_filename=se[0]
        filename=data[0]
        if se_filename==filename:
#             print(se_filename)
            data=data[1:]
            se_data=se[1:]
            out=list(map(float,data))
            out=torch.tensor(out)
            pred=model(out)
            label=list(map(float,se_data))
            label=torch.tensor(label)
            loss = criterion(pred, label)
            print_loss = loss.data.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch+=1
            if epoch%100 == 0:
            #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
            #保存参数、
                print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
                state = { 'model':model.state_dict(), 'epoch':epoch }                       
                if not os.path.isdir('checkpointX2E'):
                     os.mkdir('checkpointX2E')    
                torch.save(state, '.\checkpointX2E\modelL1LNO_epoch_%d.ckpt' % (epoch))
                file_save = open('lossX2EL1LNO.txt',mode='a')
                file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
                file_save.close() 
        else:continue
        j=j+1

0
epoch: 100, loss: 7.104
epoch: 200, loss: 6.163
epoch: 300, loss: 3.589
epoch: 400, loss: 6.505
epoch: 500, loss: 6.368
epoch: 600, loss: 5.158
epoch: 700, loss: 5.012
epoch: 800, loss: 5.019
epoch: 900, loss: 6.93
epoch: 1000, loss: 6.826
epoch: 1100, loss: 6.935
epoch: 1200, loss: 5.77
epoch: 1300, loss: 6.204
epoch: 1400, loss: 6.138
epoch: 1500, loss: 5.985
epoch: 1600, loss: 3.672
epoch: 1700, loss: 7.574
epoch: 1800, loss: 5.158
1
epoch: 1900, loss: 5.036
epoch: 2000, loss: 5.227
epoch: 2100, loss: 3.863
epoch: 2200, loss: 6.745
epoch: 2300, loss: 3.833
epoch: 2400, loss: 5.709
epoch: 2500, loss: 3.641
epoch: 2600, loss: 5.151
epoch: 2700, loss: 3.087
epoch: 2800, loss: 8.283
epoch: 2900, loss: 3.589
epoch: 3000, loss: 5.418
epoch: 3100, loss: 3.829
epoch: 3200, loss: 5.755
epoch: 3300, loss: 7.526
epoch: 3400, loss: 5.59
epoch: 3500, loss: 6.238
epoch: 3600, loss: 4.167
epoch: 3700, loss: 5.263
2
epoch: 3800, loss: 7.743
epoch: 3900, loss: 5.727
epoch: 4000, loss: 6.928
epoch:

epoch: 32000, loss: 4.521
epoch: 32100, loss: 5.189
epoch: 32200, loss: 7.704
epoch: 32300, loss: 5.258
epoch: 32400, loss: 4.705
epoch: 32500, loss: 6.09
epoch: 32600, loss: 4.421
18
epoch: 32700, loss: 4.81
epoch: 32800, loss: 3.607
epoch: 32900, loss: 4.855
epoch: 33000, loss: 6.023
epoch: 33100, loss: 3.58
epoch: 33200, loss: 8.568
epoch: 33300, loss: 5.97
epoch: 33400, loss: 5.0
epoch: 33500, loss: 4.767
epoch: 33600, loss: 6.202
epoch: 33700, loss: 6.069
epoch: 33800, loss: 4.571
epoch: 33900, loss: 5.955
epoch: 34000, loss: 3.938
epoch: 34100, loss: 3.409
epoch: 34200, loss: 4.857
epoch: 34300, loss: 6.457
epoch: 34400, loss: 7.766
19
epoch: 34500, loss: 6.195
epoch: 34600, loss: 4.467
epoch: 34700, loss: 5.795
epoch: 34800, loss: 6.159
epoch: 34900, loss: 5.004
epoch: 35000, loss: 5.968
epoch: 35100, loss: 6.437
epoch: 35200, loss: 5.82
epoch: 35300, loss: 4.114
epoch: 35400, loss: 6.476
epoch: 35500, loss: 4.483
epoch: 35600, loss: 4.75
epoch: 35700, loss: 5.343
epoch: 35800, 

epoch: 63500, loss: 4.556
epoch: 63600, loss: 3.226
epoch: 63700, loss: 4.945
epoch: 63800, loss: 5.633
epoch: 63900, loss: 3.799
epoch: 64000, loss: 7.844
epoch: 64100, loss: 3.78
35
epoch: 64200, loss: 6.645
epoch: 64300, loss: 5.598
epoch: 64400, loss: 3.544
epoch: 64500, loss: 3.846
epoch: 64600, loss: 6.805
epoch: 64700, loss: 3.883
epoch: 64800, loss: 4.135
epoch: 64900, loss: 5.739
epoch: 65000, loss: 5.631
epoch: 65100, loss: 6.109
epoch: 65200, loss: 6.673
epoch: 65300, loss: 3.634
epoch: 65400, loss: 7.218
epoch: 65500, loss: 5.272
epoch: 65600, loss: 7.276
epoch: 65700, loss: 4.637
epoch: 65800, loss: 4.677
epoch: 65900, loss: 2.529
36
epoch: 66000, loss: 6.351
epoch: 66100, loss: 4.546
epoch: 66200, loss: 5.983
epoch: 66300, loss: 6.281
epoch: 66400, loss: 5.663
epoch: 66500, loss: 4.152
epoch: 66600, loss: 6.078
epoch: 66700, loss: 4.58
epoch: 66800, loss: 4.429
epoch: 66900, loss: 6.378
epoch: 67000, loss: 6.348
epoch: 67100, loss: 5.656
epoch: 67200, loss: 5.766
epoch: 6

epoch: 95000, loss: 6.179
epoch: 95100, loss: 5.648
epoch: 95200, loss: 3.967
epoch: 95300, loss: 3.277
epoch: 95400, loss: 6.037
epoch: 95500, loss: 5.09
epoch: 95600, loss: 3.997
epoch: 95700, loss: 4.926
53
epoch: 95800, loss: 4.448
epoch: 95900, loss: 7.609
epoch: 96000, loss: 3.915
epoch: 96100, loss: 5.331
epoch: 96200, loss: 5.836
epoch: 96300, loss: 5.616
epoch: 96400, loss: 4.243
epoch: 96500, loss: 4.893
epoch: 96600, loss: 4.487
epoch: 96700, loss: 8.322
epoch: 96800, loss: 4.803
epoch: 96900, loss: 4.44
epoch: 97000, loss: 4.748
epoch: 97100, loss: 5.479
epoch: 97200, loss: 2.57
epoch: 97300, loss: 5.109
epoch: 97400, loss: 4.532
epoch: 97500, loss: 4.535
54
epoch: 97600, loss: 5.194
epoch: 97700, loss: 8.241
epoch: 97800, loss: 5.662
epoch: 97900, loss: 6.787
epoch: 98000, loss: 7.115
epoch: 98100, loss: 5.54
epoch: 98200, loss: 6.087
epoch: 98300, loss: 4.694
epoch: 98400, loss: 5.043
epoch: 98500, loss: 4.172
epoch: 98600, loss: 5.327
epoch: 98700, loss: 6.707
epoch: 988

68
epoch: 125500, loss: 3.501
epoch: 125600, loss: 4.006
epoch: 125700, loss: 5.612
epoch: 125800, loss: 7.347
epoch: 125900, loss: 3.975
epoch: 126000, loss: 5.386
epoch: 126100, loss: 7.733
epoch: 126200, loss: 7.594
epoch: 126300, loss: 3.254
epoch: 126400, loss: 4.354
epoch: 126500, loss: 7.319
epoch: 126600, loss: 4.166
epoch: 126700, loss: 5.771
epoch: 126800, loss: 3.809
epoch: 126900, loss: 3.885
epoch: 127000, loss: 5.852
epoch: 127100, loss: 5.769
epoch: 127200, loss: 4.207
epoch: 127300, loss: 5.413
epoch: 127400, loss: 5.267
69
epoch: 127500, loss: 5.735
epoch: 127600, loss: 3.591
epoch: 127700, loss: 5.586
epoch: 127800, loss: 5.138
epoch: 127900, loss: 4.841
epoch: 128000, loss: 6.483
epoch: 128100, loss: 6.363
epoch: 128200, loss: 9.677
epoch: 128300, loss: 6.047
epoch: 128400, loss: 8.063
epoch: 128500, loss: 6.399
epoch: 128600, loss: 5.001
epoch: 128700, loss: 5.333
epoch: 128800, loss: 4.931
epoch: 128900, loss: 5.011
epoch: 129000, loss: 5.577
epoch: 129100, loss: 4

In [28]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpointX2E\modelL1LNO_epoch_148600.ckpt')
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [30]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\X2E\pv_xvector.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
#     sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
#     sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

id10270-5r0dWxy17C8-00001
Synthesizing 预测语音...
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the con

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.6988470257245156


id10271-1gtz-CUIygI-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.8097465524761206


id10272-0qqQJVFKpbA-00001
Synthesizing 预测语音...


...

id10275-CVUXDNZzcmA-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.7541342151008852


id10276-3vWez3baO60-00001
Synthesizing 预测语音...


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
Synthesizing 原始语音...
阈值： 0.7131810589050301


KeyboardInterrupt: 